In [7]:
# %pip install numpy
%pip install polars


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/30.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.5 MB 330.3 kB/s eta 0:01:33
   ---------------------------------------- 0.0/30.5 MB 393.8 kB/s eta 0:01:18
   ---------------------------------------- 0.1/30.5 MB 558.5 kB/s eta 0:00:55
   ---------------------------------------- 0.1/30.5 MB 714.4 kB/s eta 0:00:43
   ---------------------------------------- 0.2/30.5 MB 1.2 MB/s eta 0:00:27
   ---------------------------------------- 0.3/30.5 MB 1.3 MB/s eta 0:00:24
    --------------------------------------- 0.6/30.5 MB 1.8 MB/s eta 0:00:17
    --------------------------------------- 0.7/30.5 MB 2.0 MB/s eta 0:00:15
   - -------------------------------------- 0.8/30.5 MB 2.2 MB/s eta 0:00:14
   - -------------------------------------- 0.8/30.5 MB 2.2 MB/s eta 0:00:14
   - -------------------------------------- 1.4/30.5 MB 2.8 MB/s eta 0:00:11
   - -------------------------------------- 1.4/30.5 MB 2.7 MB/s eta 0:00:

In [10]:
# Tristan Nguyen
# @author: tristan (NGUYEN VU ANH TRUNG)
# email = "kivanolai@gmail.com"

import numpy as np # linear algebra
import polars as pl

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('..\input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<>:12: SyntaxWarning: invalid escape sequence '\i'
<>:12: SyntaxWarning: invalid escape sequence '\i'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_8532\2505377097.py:12: SyntaxWarning: invalid escape sequence '\i'
  for dirname, _, filenames in os.walk('..\input'):


..\input\Loan_status_2007-2020Q3.csv
